In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [3]:
#this is the same code we used for a CNN predicting a number based on a handmade drawing of it
#for this, i forked another repository: https://github.com/JosePabloGFnl/exportacion-numeros

import tensorflow as tf
import tensorflow_datasets as tfds

#MNIST data set
datos, metadatos = tfds.load('mnist', as_supervised=True, with_info=True)

datos_entrenamiento, datos_pruebas = datos['train'], datos['test']

nombres_clases = metadatos.features['label'].names

#normalizing function, makes it faster
def normalizar(imagenes, etiquetas):
  imagenes = tf.cast(imagenes, tf.float32)
  imagenes /= 255 #0-255 to 0-1
  return imagenes, etiquetas

datos_entrenamiento = datos_entrenamiento.map(normalizar)
datos_pruebas = datos_pruebas.map(normalizar)

datos_entrenamiento = datos_entrenamiento.cache()
datos_pruebas = datos_pruebas.cache()

#creating the model
modelo = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax) 
])

#Compiling the model
modelo.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

num_ej_entrenamiento = metadatos.splits["train"].num_examples
num_ej_pruebas = metadatos.splits["test"].num_examples

TAMANO_LOTE = 32

#shuffling
datos_entrenamiento = datos_entrenamiento.repeat().shuffle(num_ej_entrenamiento).batch(TAMANO_LOTE)
datos_pruebas = datos_pruebas.batch(TAMANO_LOTE)

import math

#training
historial = modelo.fit(datos_entrenamiento, epochs=5, steps_per_epoch= math.ceil(num_ej_entrenamiento/TAMANO_LOTE))

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
#exporting the model to h5
modelo.save('numeros.h5')

In [ ]:
!ls

In [ ]:
!pip install tensorflow js

In [ ]:
#folder with output files
!mkdir carpeta_salida

In [ ]:
#export to output folder
!tensorflowjs_converter --input_format keras numeros.h5 carpeta_salida

In [ ]:
!ls carpeta_salida

In [ ]:
#note, if tensorflow does not work in the page, do the following:
#open cmd > cd [folder] > python -m http.server 8000
#in the address, localhost:8000/index.html